# Curvatur computation from metric tensor on parameter domain

* Evan S. Gawlik: High-order approximation of Gaussian Curvature with Regge finite elements, SINUM '20
* Yakov Berchenko-Kogan and Evan S. Gawlik: Finite element approximation of the Levi-Civita connection and its curvature in two dimensions, arXiv 2111.02512
* J. Gopalakrishnan, M. Neunteufel, JS, M. Wardetzky: Analysis of curvature approximation via covariant curl and incompatibility for Regge metrics, arXiv 2206.09343

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import CSG2d, Circle
from math import pi

order    = 4
addorder = 2

geo = CSG2d()
circle = Circle(center=(0,0), radius=1.0)
geo.Add(circle)

mesh = Mesh(geo.GenerateMesh(maxh=0.1))
mesh.Curve(10)
Draw(mesh);

In [ ]:
gfset = GridFunction(H1(mesh,order=order)**3)

r = sqrt(x**2+y**2)
phi = atan2(y,x)

gfset.Interpolate( (cos(phi)*sin(r*pi/2)-x,sin(phi)*sin(r*pi/2)-y,cos(r*pi/2)))

gfsetxy = GridFunction(H1(mesh,order=order)**2)
gfsetxy.Set ( gfset[0:2] )
mesh.SetDeformation(gfsetxy)
Draw (gfset[2], mesh, deformation=True)
mesh.UnsetDeformation()

Interpolating metric
$$
C = \nabla \varphi^T \nabla \varphi
$$
into a Regge finite element space:

In [ ]:
F = CF( (1,0, 0,1, 0,0),dims=(3,2)) + Grad(gfset)
C = F.trans*F

fesCC = HCurlCurl(mesh, order=3) 
gfC = GridFunction(fesCC)
gfC.Set(C, dual=True, bonus_intorder=4)   

In [ ]:
fesR = H1(mesh, order=3)
fR = LinearForm(fesR)
u,v = fesR.TnT()

       
# vertex - contributions = angle deficit
vertextang  = specialcf.VertexTangentialVectors(2)
vt1 = vertextang[:,0]
vt2 = vertextang[:,1]

fR += v*acos(gfC[vt1,vt2]/sqrt(gfC[vt1,vt1]*gfC[vt2,vt2]))*dx(element_vb=BBND)
fR += -v*acos((vt1*vt2)/sqrt((vt1*vt1)*(vt2*vt2)))*dx(element_vb=BBND)

t  = specialcf.tangential(2,consistent=True)
n = specialcf.normal(2)  
edgecurve = specialcf.EdgeCurvature(2)
Gamma_tt = gfC.Operator("christoffel2")[t,t,:]
fR += v*sqrt(Det(gfC))/gfC[t,t]*n*(Gamma_tt+edgecurve)*dx(element_vb=BND, bonus_intorder=4)


Riemann = gfC.Operator("Riemann")
fR += 1/sqrt(Det(gfC))*Riemann[0,1,0,1] * v * dx(bonus_intorder=4)

fR.Assemble()

gfR = GridFunction(fesR)
mass = BilinearForm(sqrt(Det(gfC))*u*v*dx(bonus_intorder=4)).Assemble().mat
gfR.vec.data = mass.Inverse() * fR.vec
Draw (gfR, min=0.99, max=1.01);

In [ ]:
print ("total curvature on half-sphere:", Integrate (gfR, mesh) / pi)